In [2]:
import gym
import random, math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
torch.manual_seed(0)

import base64, io

# For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob
from scripts.reinforce_PPORLHF import reinforce_rwd2go_PPO_RLHF
from scripts.utils import pref_save, pref_load

from scripts.model import getPolicy
import pickle

%load_ext autoreload
%autoreload 2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
SEED=0
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

ENV_NAME = 'Acrobot-v1'

env = gym.make(ENV_NAME)

K = 100

c:\Users\wahid\anaconda3\envs\BeRL\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\wahid\anaconda3\envs\BeRL\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
policy1 = getPolicy(ENV_NAME=ENV_NAME).to(device)
policy1.load_state_dict(torch.load(f"policies/policy1.pth"))
policy2 = getPolicy(ENV_NAME=ENV_NAME).to(device)
policy2.load_state_dict(torch.load(f"policies/policy2_with_mean.pth"))

pref_data = pref_load(f"pref_data/pref_data_{K}_{ENV_NAME}.pickle")

In [5]:
class RewardModel(nn.Module):
    def __init__(self, state_size=4, action_size=1, hidden_size=32):
        super(RewardModel, self).__init__()
        self.fc1 = nn.Linear(state_size + action_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        return x

    def predict_reward(self, state, action):
        state = state
        action = action
        
        state_action = torch.cat((state, action), dim=1)
        reward = self.forward(state_action).cpu()
        return reward

In [7]:
import torch
import torch.nn.functional as F
import copy

lr        = 3e-3
epochs    = 2

reward_model = RewardModel(state_size=6, action_size=1).to(device)

optimizer = torch.optim.Adam(reward_model.parameters(), lr=lr)

def trajectory_reward(reward_model, states, actions):
    total_reward = torch.tensor(0., device=device)
    for s, a in zip(states, actions):
        s_t = torch.tensor(s, dtype=torch.float32, device=device).unsqueeze(0)
        a_t = torch.tensor(data=a, dtype=torch.float32, device=device).reshape(1, -1).to(device)
        reward = reward_model.predict_reward(s_t, a_t).squeeze(0)
        total_reward += reward.squeeze(0)
    return total_reward

for epoch in range(1, epochs+1):
    total_loss = 0.0
    
    for s0, tau_plus, tau_minus in pref_data:

        reward_plus = trajectory_reward(reward_model, tau_plus["states"], tau_plus["actions"])
        reward_minus = trajectory_reward(reward_model, tau_minus["states"], tau_minus["actions"])
        
        stacked = torch.stack([reward_plus, reward_minus])
        log_Z   = torch.logsumexp(stacked, dim=0)
        total_loss += - (reward_plus - log_Z) 

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    avg_loss = total_loss / len(pref_data)
    
    print(f"Epoch {epoch}/{epochs} — avg loss: {avg_loss:.4f}")

Epoch 1/2 — avg loss: 37.8090
Epoch 2/2 — avg loss: 25.4797


In [8]:
policy2 = getPolicy(ENV_NAME=ENV_NAME).to(device)
policy2.load_state_dict(torch.load("policies/policy2_with_max.pth"))
opt1    = optim.Adam(policy2.parameters(), lr=1e-3)
reward_model.eval()
reinforce_rwd2go_PPO_RLHF(env, policy2, opt1, reward_model, n_episodes=2000);

c:\Users\wahid\anaconda3\envs\BeRL\lib\site-packages\gym\utils\passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Ep 100	avg100: -308.96
Ep 200	avg100: -394.24


KeyboardInterrupt: 

In [ ]:
returns = []
eval_episodes = 100
for ep in range(eval_episodes):
    state, done, total_r = env.reset(), False, 0.0
    while not done:
        # choose greedy or stochastic—here greedy
        with torch.no_grad():
            s_t = torch.tensor(state, dtype=torch.float32, device=device)
            probs = policy2(s_t.unsqueeze(0)).squeeze(0)
            action = torch.argmax(probs).item()
        state, r, done, _ = env.step(action)
        total_r += r
    returns.append(total_r)

mean_return = sum(returns) / len(returns)
print(f"Evaluation over {eval_episodes} episodes: mean return = {mean_return:.2f}")

Evaluation over 100 episodes: mean return = 200.00
